Add emotion in every sentence using pre-trained BERT model

Reference: [44] GoEmotions Pytorch

https://github.com/monologg/GoEmotions-pytorch

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Read an Excel file
import pandas as pd
# This must be changed by the user.
# Please change to the target video name and emotion category
video_name = 'video14'
emotion_category = 'group'
##########################################

import_excel_file_location = '/content/drive/MyDrive/05_workspace/01_msc_project/data/output/03_adding_time_info/03_adding_time_information_' + video_name +'.xlsx'
input_book = pd.ExcelFile(import_excel_file_location)


In [ ]:
# Get sheet name
input_sheet_name = input_book.sheet_names
# Get the number of sheets
num_sheet = len(input_sheet_name)

print ("The number of sheet:", num_sheet)
print (input_sheet_name)


The number of sheet: 1
['Sheet1']


In [ ]:
# Read the sheet as a DataFrame
input_sheet_df = input_book.parse(input_sheet_name[0])


In [ ]:
input_sheet_df.head(10)


,Unnamed: 0,Unnamed: 0.1,start,script,end,start_s,end_s,diff_s
0,0,0,00:02:00,my name is Donna SH and my full-time job,00:05:00,2,5,3
1,1,1,00:05:00,is to count people's arms and us and,00:08:00,5,8,3
2,2,2,00:08:00,like it all started at my first day of,00:11:00,8,11,3
3,3,3,00:11:00,school after my family moved from,00:14:00,11,14,3
4,4,4,00:14:00,Pakistan to Tanzania not from milan,00:16:00,14,16,2
5,5,5,00:16:00,I was busy sketching my dream cars in my,00:20:00,16,20,4
6,6,6,00:20:00,notebook and all of a sudden I hear a,00:22:00,20,22,2
7,7,7,00:22:00,voice in my direction the teacher,00:25:00,22,25,3
8,8,8,00:25:00,Daanish can you please rise up and,00:28:00,25,28,3
9,9,9,00:28:00,introduce yourself she had put me on the,00:31:00,28,31,3


In [ ]:
script_df = input_sheet_df['script']
len_script_df = len(script_df)
print(len_script_df)


167


In [ ]:
# Setup for GoEmotions
!pip install torch==1.4.0
!pip install transformers==2.11.0
!pip install attrdict==2.0.1


In [ ]:
# Add a path to use the module
import sys
sys.path.append('/content/drive/MyDrive/05_workspace/01_msc_project/GoEmotions-pytorch-master')


In [ ]:
# Setup for GoEmotions
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline 
from pprint import pprint

from typing import Union, Optional
import numpy as np


In [ ]:
# Create an instance of the tokenizer and model for ekman categories
if emotion_category == 'ekman':
    tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
    model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
elif emotion_category == 'group':
    tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-group")
    model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-group")


In [ ]:
# Create a pipeline
goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3
)


In [ ]:
# Add new columns
input_sheet_df['emotion'] = 0
input_sheet_df['score'] = 0.0

# Copy dataframe  
emotion_df = input_sheet_df.copy()
emotion_df.head(10)


,Unnamed: 0,Unnamed: 0.1,start,script,end,start_s,end_s,diff_s,emotion,score
0,0,0,00:02:00,my name is Donna SH and my full-time job,00:05:00,2,5,3,0,0.0
1,1,1,00:05:00,is to count people's arms and us and,00:08:00,5,8,3,0,0.0
2,2,2,00:08:00,like it all started at my first day of,00:11:00,8,11,3,0,0.0
3,3,3,00:11:00,school after my family moved from,00:14:00,11,14,3,0,0.0
4,4,4,00:14:00,Pakistan to Tanzania not from milan,00:16:00,14,16,2,0,0.0
5,5,5,00:16:00,I was busy sketching my dream cars in my,00:20:00,16,20,4,0,0.0
6,6,6,00:20:00,notebook and all of a sudden I hear a,00:22:00,20,22,2,0,0.0
7,7,7,00:22:00,voice in my direction the teacher,00:25:00,22,25,3,0,0.0
8,8,8,00:25:00,Daanish can you please rise up and,00:28:00,25,28,3,0,0.0
9,9,9,00:28:00,introduce yourself she had put me on the,00:31:00,28,31,3,0,0.0


In [ ]:
delele_list = []


In [ ]:
# (Step1) Get emotions from text with GoEmotions
# (Step2) If more than one emotion is obtained, the one with the highest socre is used
# (Step3) Append the emotion and score obtained to the appropriate columns
for s in range(len_script_df):
    
    result = []
    emotions = []
    scores = []
    
    result = goemotions(script_df[s])
    
    for list_item in result:
        if "scores" in list_item:
            scores += list_item["scores"]  
        if "labels" in list_item:
            emotions += list_item["labels"]
    
    max_score = 0
    index = -1
    
    for score in scores:
        if score > max_score:
            #print(score)
            max_score = score
            index += 1
    #print(s)
    if not emotions:
        delele_list.append(s)
        continue
    #print(emotions[index])
    #print(scores[index])
    emotion_df.loc[s,"emotion"] = emotions[index]
    emotion_df.loc[s,"score"] = scores[index]
   

In [ ]:
delele_list

[]

In [ ]:
#delele_list
emotion_df = emotion_df.drop(delele_list, axis=0)
emotion_df.head()


,Unnamed: 0,Unnamed: 0.1,start,script,end,start_s,end_s,diff_s,emotion,score
0,0,0,00:02:00,my name is Donna SH and my full-time job,00:05:00,2,5,3,neutral,0.999663
1,1,1,00:05:00,is to count people's arms and us and,00:08:00,5,8,3,neutral,0.999567
2,2,2,00:08:00,like it all started at my first day of,00:11:00,8,11,3,neutral,0.999066
3,3,3,00:11:00,school after my family moved from,00:14:00,11,14,3,neutral,0.998456
4,4,4,00:14:00,Pakistan to Tanzania not from milan,00:16:00,14,16,2,neutral,0.999349


In [ ]:
# Setup for exporting to Excel
!pip install -U openpyxl
!pip install -U xlwt
#!pip install -U pandas


In [ ]:
# Export an excel file
import pandas as pd

export_excel_file_location = '/content/drive/MyDrive/05_workspace/01_msc_project/data/output/04_emotion_extraction_from_subtitles/' + emotion_category + '/04_emotion_extraction_' + video_name +'.xlsx'
emotion_df.to_excel(export_excel_file_location)


In [ ]:
# Counting for each emotional category
vc = emotion_df['emotion'].value_counts()
print(vc[:])


neutral      134
positive      16
negative      12
ambiguous      5
Name: emotion, dtype: int64
